# 无人驾驶

* 模型简介：
    该模型使用机器人的摄像头并通过目标检测进行车道检测，斑马线识别，标识牌识别，红绿灯识别，行人识别，障碍检测来模拟汽车在现实生活中的无人驾驶。
    
![title](other_data/01.jpg)

* 模型最佳运行环境：室内。

* 模型所需材料：机器人1台、地图一张，交通标识牌一批，红绿灯两个。


## 1.导入我们需要的模块

In [ ]:
# from __future__ import division
from nxbot import Robot,event,bgr8_to_jpeg

from modules.models import *
from modules.utils.util import *
from modules.utils.datasets import *
import os
import sys
import argparse
import cv2
import threading
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from torch2trt import TRTModule

## 2.参数设置

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--model_def", type=str, default="modules/config/auto_drive.cfg", help="yolov3-tiny网络结构配置文件")
parser.add_argument("--Dachbot_model_path", type=str, default="../../../models/local/yolov3/auto_drive.pth", help="Dachbot模型文件")
parser.add_argument("--Dbot_model_path", type=str, default="../../../models/local/yolov3/auto_drive.engine", help="Dbot模型文件")
parser.add_argument("--class_path", type=str, default="modules/data/custom/classes.names", help="检测类别的所有种类")
parser.add_argument("--conf_thres", type=float, default=0.9, help="物体置信度")
parser.add_argument("--nms_thres", type=float, default=0.4, help="非极大抑制阈值")
parser.add_argument("--img_size", type=int, default=416, help="网络接收图片大小")
opt = parser.parse_args(args=[])

## 3.加载模型

In [ ]:
# 实例化机器人对象
rbt = Robot()
# 机器人名字
rbt_name = rbt.name
print(rbt_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 如果是dachbot可以使用更准确的模型
if rbt_name=='dachbot':
    model = Darknet(opt.model_def, img_size=opt.img_size, TensorRT=False, Half=False).to(device).half()
    # 权重加载
#     model.load_darknet_weights(opt.Dachbot_model_path)
    model.load_state_dict(torch.load(opt.Dachbot_model_path))
    # Set in evaluation mode 前向推理时候会忽略 BatchNormalization 和 Dropout
    model.eval()
    
# 如果是dbot可以使用速度更快的模型    
elif rbt_name=='dbot':
    model_backbone = Darknet_Backbone(opt.model_def, img_size=opt.img_size).to(device).half()
    model = TRTModule()
    model.load_state_dict(torch.load(opt.Dbot_model_path))
    yolo_head = YOLOHead(config_path=opt.model_def)
    
# 提取可以识别的类别
classes = load_classes(opt.class_path)  # Extracts class labels from file
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

## 4.创建可视化窗口

In [ ]:
image_widget = widgets.Image(format='jpeg')
depth_image_widget = widgets.Image(format='jpeg')
speed_widget = widgets.FloatSlider(value=0.16, min=0.0, step=0.01, max=0.5, description='运行速度')
turn_gain_widget = widgets.FloatSlider(value=1.6, min=0.0, step=0.01, max=3.0, description='转向增益')
turn_dgain_widget = widgets.FloatSlider(value=0.03, min=0.0, step=0.01, max=5.0, description='回正微调')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='方向')
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='深度值')
state_info = widgets.Textarea(
    placeholder='NXROBO',
    description='当前状态',
    disabled=False
)

## 5.定义目标检测
在这里我们将让机器人识别出它所看到的所有物体，并且判断距离图像中心点最近的物体是否与我们设定的目标物体一致（我们默认的是索引1，代表跟踪的目标是人），如果是设定目标，就跟着目标走，并根据物体再图像的位置让机器人判断左转还是右转。

### 5.1.计算识别的物体的中心点坐标相对于图片中心点的距离

In [ ]:
def detection_center(label):
    dis_x = 0.5-((label[2] - label[0])/2+label[0])
    return dis_x

### 5.3.找出最适合的线进行巡线
* 找出所有线中右下角y轴坐标大于120中y轴坐标值最大的线

In [ ]:
def closest_line(matching_lines):
    closest_det = None
    for det in matching_lines:
        center_x = (det[2]-det[0])/2+det[0]
        if det[3]>100:
            if det[3]<150:
                if center_x>120 and center_x<180:
                    if closest_det is None:
                        closest_det = det
                    elif det[3]>closest_det[3]:
                        closest_det = det
            else:
                if center_x>20 and center_x<280:
                    if closest_det is None:
                        closest_det = det
                    elif det[3]>closest_det[3]:
                        closest_det = det
    return closest_det

## 6.数据预处理

In [ ]:
def preprocess(image):
    image = np.array(Image.fromarray(cv2.cvtColor(image,cv2.COLOR_BGR2RGB)))
    imgTensor = transforms.ToTensor()(image)
    imgTensor, _ = pad_to_square(imgTensor, 0)
    imgTensor = resize(imgTensor, 416)
    imgTensor = imgTensor.unsqueeze(0)
    imgTensor = Variable(imgTensor.type(Tensor)).half()
    return imgTensor

## 7.检测模型是否能正常运行

我们通过numpy创建与我们将要预测的图片格式一致的形状为（416，416，3）的数组，这里我们创建的全为1的数组将这个数组经过预处理再将数据放入模型中，如果能运行通过说明模型可以正常使用了。

In [ ]:
try:
    img_data = np.ones([416, 416, 3],np.uint8)
    if rbt_name=='dachbot':
        model(preprocess(img_data)).detach().half().cpu().numpy().flatten()
    elif rbt_name=='dbot':
        model(preprocess(img_data))
except:
    print('请检查模型是否正确')

## 8.机器人目标检测
我们通过字典存储机器人看到的物体以以及对应物体的次数，我们可以通过下面的函数去过滤太小的目标。

In [ ]:
def change_state(origin_img, detections, signal_dict):
    max_signal_size = 0
    max_classes = None
    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
        label_name = classes[int(cls_pred)]
        if label_name!='line':
            box_w = x2 - x1
            box_h = y2 - y1
            color = [int(c) for c in colors[int(cls_pred)]]
            origin_img = cv2.rectangle(origin_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(origin_img, label_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(origin_img, str("%.2f" % float(conf)), (x2, y2 - box_h), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        color, 2)
            
            box_size = box_w*box_h
            if label_name == 'aobing':
                if box_size>500:
                    signal_dict['aobing']+=1

            if label_name == 'nezha':
                if box_size>500:
                    signal_dict['nezha']+=1

            if label_name == 'zebra_crossing':
                signal_dict['zebra_crossing_y']= y2
                if abs(150-box_w/2+x1)<80 and box_w/box_h>1.2:
                    signal_dict['zebra_crossing']+=1
                    
            if label_name not in ['aobing','nezha','zebra_crossing']:
                if label_name in ['car_red','car_red_yellow','car_green','car_yellow']:
                    if box_size>2000:
                        if label_name == 'car_red':
                            signal_dict['car_red']+=1
                        if label_name == 'car_red_yellow':
                            signal_dict['car_red_yellow']+=1
                        if label_name == 'car_green':
                            signal_dict['car_green']+=1
                        if label_name == 'car_yellow':
                            signal_dict['car_yellow']+=1
                else:
                    if box_size>3500:
                        if box_size>max_signal_size:
                            max_signal_size = box_size
                            max_classes = label_name
                        if max_classes is not None:
                            if max_classes == 'forward':
                                signal_dict['forward']+=1
                            if max_classes == 'left':
                                signal_dict['left']+=1
                            if max_classes == 'right':
                                signal_dict['right']+=1
                            if max_classes == 'turn':
                                signal_dict['turn']+=1
                            if max_classes == 'stop':
                                signal_dict['stop']+=1
    return signal_dict

## 9.定义图像检测
我们使用下面这个线程调用目标检测模型进行识别，然后将识别到的物体通过上面的函数进行过滤，并保存在定义的字典当中。

In [ ]:

# 随机选择颜色
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype="uint8")

init_state = {'find_line':0,'lights_off':0,'car_red':0,'car_yellow':0,'car_green':0,'car_red_yellow':0,
              'pedestrian_red':0,'pedestrian_green':0,'nezha':0,'aobing':0,'line':0,'zebra_crossing':0,
              'zebra_crossing_y':0, 'forward':0,'left':0,'right':0,'turn':0,'stop':0}


global signal_dict
signal_dict = init_state
def prediction():
    global detect_flag
    detect_flag = True
    
    global matching_line
    matching_line = None
    
    aobing_list = []
    nezha_list = []
    
    while detect_flag:
        
        global signal_dict
        time.sleep(0.01)
        origin_img = rbt.camera.read()
        detections = None
        if origin_img is not None:
            # 对图像数据进行预处理
            imgTensor = preprocess(origin_img)

            with torch.no_grad():
                # 将图像数据放入模型进行预测
                outputs = model(imgTensor)
                # 如果是dbot采用另外一种方法
                if rbt_name=='dbot':
                    outputs = yolo_head(outputs)
                # 非极大抑制筛选更加合适的候选框
                outputs = non_max_suppression(outputs, opt.conf_thres, opt.nms_thres)[0]
                
            line_list = []
            if outputs is not None:
                
                aobing_list = []
                nezha_list = []
                # 对预测类别框进行缩放
                detections = rescale_boxes(outputs, opt.img_size, origin_img.shape[:2])
                # 记录识别到的物体
                signal_dict = change_state(origin_img, detections, signal_dict)

                line_list = [d for d in detections if classes[int(d[6])] == 'line']
                aobing_list = [d for d in detections if classes[int(d[6])] == 'aobing']
                nezha_list = [d for d in detections if classes[int(d[6])] == 'nezha']
                        
            if aobing_list==[]:
                signal_dict['aobing']=0
            if nezha_list==[]:
                signal_dict['nezha']=0
                
            if line_list != []:
                # 找出跟踪的那一条line
                matching_line = closest_line(line_list)
                if matching_line is not None:
                    signal_dict['find_line']+=1
                    # 用红色标记标记目标line
                    cv2.rectangle(origin_img, (matching_line[0], matching_line[1] + (matching_line[3]-matching_line[1])), 
                                               (matching_line[2], matching_line[1]), (0,0,255), 2)
                else:
                    signal_dict['find_line'] = 0
                        
            origin_img = cv2.resize(origin_img, (320, 240), interpolation=cv2.INTER_CUBIC)
            image_widget.value=bgr8_to_jpeg(origin_img)

# 创建线程
process1 = threading.Thread(target=prediction,)
# 启动线程
process1.start()

## 10.定义深度值获取函数
该函数只适用于机器人dachbot，该函数通过调用机器人的深度摄像头进行深度检测

In [ ]:
global depth
depth = 0
def on_new_depth(evt):
    depth_frame = np.asanyarray(evt.dict['data'].get_data())
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_frame, alpha=0.03), cv2.COLORMAP_JET)
    depth_colormap = cv2.resize(depth_colormap, (320,240))
    depth_image_widget.value = bgr8_to_jpeg(depth_colormap)
    
    global depth
    depth = evt.dict['data'].get_distance(310, 80)
    if depth ==0:
        depth = evt.dict['data'].get_distance(330, 85)
    if depth ==0:
        depth = evt.dict['data'].get_distance(331, 83)
    depth_slider.value = depth

## 11.控制机器人运动
我们使用另外一个线程去检查目标检测记录的物体字典，通过该字典我们就能判断机器人看到了哪些物体或者标识牌，然后我们根据识别到该物体的次数让机器人做出相应的运动。
* 1.我们会优先判断前方是否有障碍，如果没有障碍再判断前方是否有行人，如果没有行人再判断是否有斑马线；
* 2.当看到斑马线时，就停下并抬头看前方是否有红绿灯；
* 3.如果有红绿灯，判断是否为绿灯，如果不是绿灯就停下不动，如果是绿灯就判断是否有标识牌，如果没有标识牌就默认直行再进行巡线，如果有标识牌就按照标识牌指示进行运动，运动完再进行巡线；
* 4.如果前方既没有斑马线也没有红绿灯就默认直行再进行巡线，如果只有标识牌就按照标识牌指示进行运动，完成之后再进行巡线；
* 5.如果什么都没有就默认巡线。

### 11.1.机器人dachbot逻辑控制定义

In [ ]:
global speech_word
speech_word=None

def dachbot_move():
    last_steering = 0
    no_line = 0
    steering = 0
    rise_head = 0
    num_line = 0
    global move_robot
    move_robot=True
    robot_state = {'road_following':True, 'signal_detection':False, 'start':True}
    first_time = time.time()
    global speech_word
    speech_word=None
    while move_robot:
        time.sleep(0.01)
        global matching_line
        global signal_dict
        global depth
        if depth!=0:
            if depth>0.3:
                if signal_dict['zebra_crossing']>2 and signal_dict['zebra_crossing_y']>260:
                    if (time.time()-first_time)>5:
                        state_info.value = '发现斑马线'
                        speech_word = '发现斑马线'
                        robot_state['road_following'] = False
                        robot_state['signal_detection'] = True
                        signal_dict['zebra_crossing']=0
                        rise_head = 0
                        num_line = 0
                        rbt.base.move(0,0,0)
                    else:
                        signal_dict['zebra_crossing']=0
                        signal_dict['zebra_crossing_y']=0
                else:
                    robot_state['road_following'] = True
                    
                
                if signal_dict['nezha']==0 and signal_dict['aobing']==0:
                    robot_state['road_following'] = True
                else:
                    robot_state['road_following'] = False
                    rbt.base.move(0,0,0)
                    if signal_dict['nezha']>5:
                        state_info.value = '看到哪吒'
                        speech_word = '发现哪吒在前方'
                    if signal_dict['aobing']>5:
                        state_info.value = '看到鳌丙'
                        speech_word = '发现鳌丙在前方'
                    
                if robot_state['road_following']==True and robot_state['signal_detection']==False:
                    if matching_line is not None:
                        robot_state['start']=False
                        target_line = matching_line/300
                        state_info.value = '找到line，正常行驶'
                        # 计算line的x轴中心点
                        distance_x = detection_center(target_line)
                        # 根据距离计算转向值
                        steering = distance_x*turn_gain_widget.value
                        steering = steering - (steering + last_steering)*turn_dgain_widget.value
                        steering_slider.value = -steering
                        speed = speed_widget.value
                        rbt.base.move(speed, 0, steering)
                        last_steering = steering
                        no_line=0
                    else:
                        no_line+=1
                        if no_line>200:
                            state_info.value = '没有找到line，停止'
                            rbt.base.move(0,0,0)
                        else:
                            if robot_state['start']==False:
                                state_info.value = '继续行驶'
                                speed = speed_widget.value
                                rbt.base.move(speed, 0, last_steering)
                if robot_state['signal_detection'] == True:
                    if rise_head == 0:
                        rbt.base.set_ptz(20)
                        time.sleep(3)
                        rise_head+=1
                    else:
                        # 如果有停止标志
                        if signal_dict['stop']>5:
                            state_info.value = '检测到停止标志'
                            speech_word = '检测到停止标志'
                            rbt.base.move(0,0,0)
                            signal_dict['stop']=0
                            rise_head = 0
                        else:
                            if signal_dict['car_red_yellow']>5:
                                state_info.value = '检测到红黄灯'
                                speech_word = '检测到红黄灯'
                                signal_dict['car_red_yellow']=0
                                rbt.base.move(0,0,0)
                                rise_head = 0
                            elif signal_dict['car_yellow']>5:
                                state_info.value = '检测到黄灯'
                                speech_word = '检测到黄灯'
                                signal_dict['car_yellow']=0
                                rbt.base.move(0,0,0)
                                rise_head = 0

                            elif signal_dict['car_red']>5:
                                state_info.value = '检测到红灯'
                                speech_word = '检测到红灯'
                                signal_dict['car_red']=0
                                rbt.base.move(0,0,0)
                                rise_head = 0
                            else:
                                # 如果没有绿灯的情况
                                if signal_dict['car_green']==0:
                                    state_info.value = '没有检测到绿灯'
                                else:
                                    state_info.value = '检测到绿灯'

                                if signal_dict['forward']>5:
                                    state_info.value = '检测到直行'
                                    speech_word = '检测到直行'
                                    rbt.base.set_ptz(-35)
                                    time.sleep(2)
                                    rbt.base.forward(0.15, 2, True)
                                    signal_dict['forward']=0

                                elif signal_dict['left']>5:
                                    state_info.value = '检测到左转'
                                    speech_word = '检测到左转'
                                    rbt.base.set_ptz(-35)
                                    time.sleep(2)
                                    if signal_dict['aobing']>3 or signal_dict['nezha']>3:
                                        rbt.base.move(0,0,0)
                                    else:
                                        rbt.base.forward(0.1, 3, True)
                                        rbt.base.move(0.12,0,0.6,3)
                                        signal_dict['left']=0

                                elif signal_dict['right']>5:
                                    state_info.value = '检测到右转'
                                    speech_word = '检测到右转'
                                    rbt.base.set_ptz(-35)
                                    time.sleep(2)
                                    if signal_dict['aobing']>3 or signal_dict['nezha']>3:
                                        rbt.base.move(0,0,0)
                                    else:
                                        rbt.base.forward(0.1, 3, True)
                                        rbt.base.move(0.12,0,-0.6,3)
                                        signal_dict['right']=0

                                elif signal_dict['turn']>5:
                                    rbt.base.set_ptz(-45)
                                    signal_dict['find_line'] = 0
                                    state_info.value = '检测到掉头'
                                    speech_word = '检测到掉头'

                                    if random.random()<0.5:
                                        turn = 'left'
                                    else:
                                        turn = 'right'
                                    start_time = time.time()
                                    while True:
                                        if time.time()-start_time<3:
                                            signal_dict['find_line'] = 0

                                        if signal_dict['find_line'] < 2:
                                            if turn=='left':
                                                rbt.base.turnleft(0.4, 1,True)
                                            else:
                                                rbt.base.turnright(0.4, 1,True)
                                        else:
                                            signal_dict['turn']=0
                                            rbt.base.set_ptz(-35)
                                            break

                                # 没有其他灯
                                else:
                                    state_info.value = '没有检测到标识牌'
                                    speech_word = '没有检测到标识牌'
                                    rbt.base.set_ptz(-35)
                                    time.sleep(2)
                                    while num_line<4:
                                        time.sleep(0.2)
                                        rbt.base.move(speed, 0, last_steering*0.5)
                                        num_line+=1
                                        
                                robot_state['signal_detection'] = False
                                robot_state['road_following'] = True
                                signal_dict = init_state
                    first_time = time.time()
            else:
                state_info.value = '前方有障碍'
                speech_word = '前方有障碍'
                rbt.base.move(0,0,0)

### 11.1.机器人dbot逻辑控制定义
机器人dbot无法检测到前方物体的距离

In [ ]:
def dbot_move():
    last_steering = 0
    no_line = 0
    steering = 0
    rise_head = 0
    num_line = 0
    global move_robot
    move_robot=True
    robot_state = {'road_following':True, 'signal_detection':False, 'start':True}
    first_time = time.time()
    global speech_word
    speech_word=None
    global rbt_name
    while move_robot:
        time.sleep(0.01)
        global matching_line
        global signal_dict
        
        if signal_dict['zebra_crossing']>2 and signal_dict['zebra_crossing_y']>260:
            if (time.time()-first_time)>5:
                state_info.value = '发现斑马线'
                speech_word = '发现斑马线'
                robot_state['road_following'] = False
                robot_state['signal_detection'] = True
                signal_dict['zebra_crossing']=0
                rise_head = 0
                num_line = 0
                rbt.base.move(0,0,0)
            else:
                signal_dict['zebra_crossing']=0
                signal_dict['zebra_crossing_y']=0
        else:
            robot_state['road_following'] = True


        if signal_dict['nezha']==0 and signal_dict['aobing']==0:
            robot_state['road_following'] = True
        else:
            robot_state['road_following'] = False
            rbt.base.move(0,0,0)
            if signal_dict['nezha']>2:
                state_info.value = '看到哪吒'
                speech_word = '发现哪吒在前方'
            if signal_dict['aobing']>5:
                state_info.value = '看到鳌丙'
                speech_word = '发现鳌丙在前方'

        if robot_state['road_following']==True and robot_state['signal_detection']==False:
            if matching_line is not None:
                robot_state['start']=False
                target_line = matching_line/300
                state_info.value = '找到line，正常行驶'
                # 计算line的x轴中心点
                distance_x = detection_center(target_line)
                # 根据距离计算转向值
                steering = distance_x*turn_gain_widget.value
                steering = steering - (steering + last_steering)*turn_dgain_widget.value
                steering_slider.value = -steering
                speed = speed_widget.value
                rbt.base.move(speed, 0, steering)
                last_steering = steering
                no_line=0
            else:
                no_line+=1
                if no_line>200:
                    state_info.value = '没有找到line，停止'
                    rbt.base.move(0,0,0)
                else:
                    if robot_state['start']==False:
                        state_info.value = '继续行驶'
                        speed = speed_widget.value
                        rbt.base.move(speed, 0, last_steering)

        if robot_state['signal_detection'] == True:
            if rise_head == 0:
                rbt.base.set_ptz(20)
                time.sleep(3)
                rise_head+=1
            else:
                # 如果有停止标志
                if signal_dict['stop']>5:
                    state_info.value = '检测到停止标志'
                    speech_word = '检测到停止标志'
                    rbt.base.move(0,0,0)
                    signal_dict['stop']=0
                    rise_head = 0
                else:
                    if signal_dict['car_red_yellow']>5:
                        state_info.value = '检测到红黄灯'
                        speech_word = '检测到红黄灯'
                        signal_dict['car_red_yellow']=0
                        rbt.base.move(0,0,0)
                        rise_head = 0
                    elif signal_dict['car_yellow']>5:
                        state_info.value = '检测到黄灯'
                        speech_word = '检测到黄灯'
                        signal_dict['car_yellow']=0
                        rbt.base.move(0,0,0)
                        rise_head = 0

                    elif signal_dict['car_red']>5:
                        state_info.value = '检测到红灯'
                        speech_word = '检测到红灯'
                        signal_dict['car_red']=0
                        rbt.base.move(0,0,0)
                        rise_head = 0
                    else:
                        # 如果没有绿灯的情况
                        if signal_dict['car_green']==0:
                            state_info.value = '没有检测到绿灯'
                        else:
                            state_info.value = '检测到绿灯'

                        if signal_dict['forward']>5:
                            state_info.value = '检测到直行'
                            speech_word = '检测到直行'
                            rbt.base.set_ptz(-35)
                            time.sleep(2)
                            rbt.base.forward(0.15, 2, True)
                            signal_dict['forward']=0

                        elif signal_dict['left']>5:
                            state_info.value = '检测到左转'
                            speech_word = '检测到左转'
                            rbt.base.set_ptz(-35)
                            time.sleep(2)
                            if signal_dict['aobing']>3 or signal_dict['nezha']>3:
                                rbt.base.move(0,0,0)
                            else:
                                rbt.base.forward(0.1, 3, True)
                                rbt.base.move(0.12,0,0.6)
                                time.sleep(2)
                                signal_dict['left']=0

                        elif signal_dict['right']>5:
                            state_info.value = '检测到右转'
                            speech_word = '检测到右转'
                            rbt.base.set_ptz(-35)
                            time.sleep(2)
                            if signal_dict['aobing']>3 or signal_dict['nezha']>3:
                                rbt.base.move(0,0,0)
                            else:
                                rbt.base.forward(0.1, 3, True)
                                rbt.base.move(0.12,0,-0.6)
                                time.sleep(2)
                                signal_dict['right']=0

                        elif signal_dict['turn']>5:
                            rbt.base.set_ptz(-45)
                            signal_dict['find_line'] = 0
                            state_info.value = '检测到掉头'
                            speech_word = '检测到掉头'

                            if random.random()<0.5:
                                turn = 'left'
                            else:
                                turn = 'right'
                            start_time = time.time()
                            while True:
                                if time.time()-start_time<3:
                                    signal_dict['find_line'] = 0

                                if signal_dict['find_line'] < 2:
                                    if turn=='left':
                                        rbt.base.turnleft(0.4, 1,True)
                                    else:
                                        rbt.base.turnright(0.4, 1,True)
                                else:
                                    signal_dict['turn']=0
                                    rbt.base.set_ptz(-35)
                                    break

                        # 没有其他灯
                        else:
                            state_info.value = '没有检测到标识牌'
                            speech_word = '没有检测到标识牌'
                            rbt.base.set_ptz(-35)
                            time.sleep(2)
                            while num_line<4:
                                time.sleep(0.2)
                                rbt.base.move(speed, 0, last_steering*0.5)
                                num_line+=1

                        robot_state['signal_detection'] = False
                        robot_state['road_following'] = True
                        signal_dict = init_state
            first_time = time.time()
if rbt_name=='dachbot':
    # 创建线程
    process2 = threading.Thread(target=dachbot_move,)
elif rbt_name=='dbot':
    process2 = threading.Thread(target=dbot_move,)
# 启动线程
process2.start()

## 12.机器人语音播报看到的物体或者标识牌
使用线程获取机器人看到的物体类别，并进行语音合成。

In [ ]:
def run_speech():
    global speech_run
    speech_run = True
    last_word = ''
    while speech_run:
        time.sleep(0.05)
        global speech_word
        if speech_word is not None:
            state_info.value = speech_word
            if last_word!= speech_word:
                rbt.speech.play_text(speech_word,True)
                last_word = speech_word
# 创建线程
process3 = threading.Thread(target=run_speech,)
# 启动线程
process3.start()

## 13.连接机器人进行实时检测

In [ ]:
# 连接机器人
rbt.connect()

# 如果机器人是dachbot
if rbt_name=='dachbot':
    rbt.camera.start(enable_depth_stream=True)
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_DEPTH,on_new_depth)
    display(widgets.HBox([image_widget, depth_image_widget]))
    display(widgets.VBox([speed_widget,turn_gain_widget, turn_dgain_widget, steering_slider, depth_slider]))

# 如果机器人是dbot
elif rbt_name=='dbot':
    rbt.camera.start()
    display(image_widget)
    display(widgets.VBox([speed_widget,turn_gain_widget, turn_dgain_widget, steering_slider]))

# 调整机器人摄像头角度
rbt.base.set_ptz(-35)

# 显示机器人当前的状态信息
display(state_info)

## 11.断开与机器人的连接

In [ ]:
# detect_flag = False
# move_robot = True
# speech_run = False
# rbt.disconnect()